In [1]:
from runner import Runner
from model_lgb import ModelLGB
import matplotlib.pyplot as plt
import pandas as pd
from dataset import Dataset

In [2]:
Dataset.get_all_feature_names()

Index(['人口', '傷病程度：その他', '傷病程度：中等症', '傷病程度：死亡', '傷病程度：軽症', '傷病程度：重症',
       '天気概況(夜)', '天気概況(昼)', '平均気温', '平均湿度', '平均現地気圧', '平均蒸気圧', '平均雲量', '平均風速',
       '年齢区分：不明', '年齢区分：乳幼児', '年齢区分：少年', '年齢区分：成人', '年齢区分：新生児', '年齢区分：高齢者',
       'target', 'date', '日照時間', '最低気温', '最高気温', '発生場所：その他', '発生場所：仕事場①',
       '発生場所：仕事場②', '発生場所：住居', '発生場所：公衆(屋内)', '発生場所：公衆(屋外)', '発生場所：教育機関',
       '発生場所：道路', '県名', '降水量', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [3]:
plt.rcParams["font.size"] = 18

In [4]:
prms = {
    # lightGBM
    'num_round': 10000,
    'early_stopping_rounds': 10,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metrics': 'huber',
    'num_leaves': 30,
    'learning_rate': 0.3,
    'verbose': 0,

    # target
    'target': 'target',

    # 街・年
    'cities': ['tokyo'],
    'train_years': [i for i in range(2010, 2017)] + [2018],
    #'train_years': [i for i in range(2010, 2015)],
    'valid_years': [2017, 2018],
    'test_years': [2018],

    # cv
    'cv': 'manual',
}

features = [
    'target','最高気温', '平均気温', '最低気温', '平均湿度',
    '平均現地気圧', '平均蒸気圧', '平均雲量', '平均風速', '日照時間', '降水量',
    'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'
]

In [5]:
%matplotlib inline
runner = Runner(run_name='test', model_cls=ModelLGB, features=features, prms=prms)
valid_score, test_score = runner.run()
print(valid_score, test_score)
runner.feature_importance()
runner.plot_prediction()

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: 3

In [ ]:
from dataset import Dataset
import matplotlib.pyplot as plt

dataset = Dataset(features, 'target')
data = dataset.get_data([i for i in range(2008, 2019)], ['tokyo'])
plt.figure(figsize=(40, 20))
plt.plot(data['target'].values)
plt.show()

In [ ]:
from dataset import Dataset
import matplotlib.pyplot as plt

dataset = Dataset(features, 'target')
data = []
for i in range(2008, 2019):
    data.append(dataset.get_data([i], ['tokyo']))
    data[-1] = data[-1]['target'].values
    print(data[-1].shape)
    plt.figure()
    plt.plot(data[-1])
    plt.show()

In [6]:
from dataset import Dataset
import matplotlib.pyplot as plt

import pandas as pd

#現在の最大表示列数の出力
pd.get_option("display.max_columns")

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

import pandas as pd

#現在の最大表示行数の出力
pd.get_option("display.max_rows")

#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 1000000)

dataset = Dataset(features, 'target')
dataset.dataset

,target,最高気温,平均気温,最低気温,平均湿度,平均現地気圧,平均蒸気圧,平均雲量,平均風速,日照時間,降水量,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,県名,date
0,0.0,25.5,21.8,18.6,71.0,1008.3,18.4,9.5,2.7,7.8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,tokyo,2008-07-01
1,2.0,27.8,24.6,22.1,70.0,1005.6,21.4,9.0,2.3,2.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,tokyo,2008-07-10
2,6.0,29.4,26.0,23.5,71.0,1001.9,23.7,8.8,3.1,5.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,tokyo,2008-07-11
3,35.0,33.8,28.0,24.1,67.0,1000.7,25.1,7.5,2.1,10.6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,tokyo,2008-07-12
4,24.0,32.7,28.5,25,65.0,1004.3,25.1,6.3,2.4,9.6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,tokyo,2008-07-13
5,16.0,32.7,28.9,26.2,69.0,1004.5,27.2,9.5,1.9,4.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,tokyo,2008-07-14
6,16.0,31.4,27.8,24.8,66.0,1005.3,24.5,10.0,3.4,4.6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,tokyo,2008-07-15
7,26.0,31.7,27.3,23.5,74.0,1003.3,26.5,9.3,2.6,6.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,tokyo,2008-07-16
8,15.0,31.7,27.9,25.3,74.0,1002.6,27.6,4.8,2.4,5.1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,tokyo,2008-07-17
9,16.0,32.1,28.7,27.1,75.0,1001.4,29.4,9.3,2.4,1.4,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,tokyo,2008-07-18
